# Converting .xml to pd.DataFrame

In [1]:
#Import der packages
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

In [2]:
#Import von .xml Daten

import xml.etree.ElementTree as ET


def read_xml(path):
   xml = ET.parse(path).getroot()
   return xml

def search_for_name(tree, attr):
   names = []
   for child in tree:
       if child.tag == attr:
           names += [child.text]
       else:
           names += search_for_name(child, attr)
   return names

#Test
if __name__=="__main__":
   xml = read_xml("MDB_STAMMDATEN.XML")
   names = search_for_name(xml,"GESCHLECHT")
   
print(names[:4])
print(len(names))

['männlich', 'männlich', 'weiblich', 'weiblich']
4073


In [3]:
#Erste Ansätze für ein DataFrame mit den folgenden Spalten

#Merkmale
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND", "WP"]

#Einmaliges zusammensetzten einer Liste von Listen mit den Ausprägungen der obigen Merkmale
column_list = []
for attr in liste_merk:
    if __name__=="__main__":
        xml = read_xml("MDB_STAMMDATEN.XML")
        names = search_for_name(xml, attr)
        column_list.append(names)

In [4]:
#Nun wird die Kompatibilität der einzelnen Listen überprüft. Es soll ein pd.DataFrame erstellt werden und dafür müssen die 
#Spaltenlängen übereinstimmen

print(len(column_list))

for i in column_list:
    print(len(i))

9
4073
4336
4336
4073
4073
4073
4073
4073
11600


In [5]:
#Mit diesen verkürzten Liste lässt sich schon ein DataFrame erstellen
liste_merk_2 = ["ID","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]

In [6]:
#Etwas umstänlich, aber wirksam wird ein DF zusammen gestellt und ein erster Mangel behoben 
#(none values werden durch "Deutschland erszetz")

df_first = pd.DataFrame(np.column_stack([column_list[0],column_list[3],column_list[4],column_list[5],column_list[6],column_list[7]]), columns = liste_merk_2)

df_first.loc[df_first["GEBURTSLAND"].isnull(), "GEBURTSLAND"] = "Deutschland"


print(df_first.head())

   

         ID GESCHLECHT PARTEI_KURZ GEBURTSDATUM  \
0  11000001   männlich         CDU         1930   
1  11000002   männlich         FDP         1909   
2  11000003   weiblich         CDU         1913   
3  11000004   weiblich         CDU         1933   
4  11000005   männlich         CDU         1950   

                                               BERUF  GEBURTSLAND  
0  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...  Deutschland  
1                             Rechtsanwalt und Notar  Deutschland  
2                                    Hilfsreferentin  Jugoslawien  
3                                             Ärztin  Deutschland  
4                      Mathematiker, Geschäftsführer  Deutschland  


In [7]:
#hier besteht noch Verbesserungsbedarf, dazu aber später mehr
df_first.dtypes

ID              object
GESCHLECHT      object
PARTEI_KURZ     object
GEBURTSDATUM    object
BERUF           object
GEBURTSLAND     object
dtype: object

In [8]:
#Mit Hilfe einer höheren Macht lässt sich der Parsing-Prozess weiterentwickeln
#Hier für die Vor- und Nachnamen

def search_for_nameRoots(root):
 names = []
 for child in root:
     if child.tag == "NAMEN":
         names += [child]
     else:
         names += search_for_nameRoots(child)
 return names

xml = read_xml(path = "MDB_STAMMDATEN.XML")

nameRoots = search_for_nameRoots(xml)

firstNameElements = []
for element in nameRoots:
   firstNameElements += [element[0].find('VORNAME').text]

lastNameElements = []
for element in nameRoots:
   lastNameElements += [element[0].find('NACHNAME').text]

df_first["NACHNAME"] = lastNameElements
df_first["VORNAME"] = firstNameElements


In [9]:
#Die Integration ist geglückt
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich


In [10]:
#Nun muss der Parsing-Prozess nochmal umgestaltet werden, um zu einer spezifischen ID/einem spezifischen MdB die Nummern
#aller Wahlperioden zu finden.
#Langfrisitg soll auf diese Perioden umindiziert werden.


#Die folgende Funktion soll eine Liste von Listen dieser nummerierten Wahlperioden produzieren

def search_for_wahlRoots1(root):
    
    #nestet function (Entscheidend ist hier der Break beim nächsten "ID" tag)
    def search_for_wpRoots2(root2):
        wplist2 = []
        for child2 in root2:
            if child2.tag == "WP":
                wplist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                wplist2 += search_for_wpRoots2(child2)
        return wplist2
    #ende nested function
    
    wplist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            wplist += [search_for_wpRoots2(child)]
        else:
            wplist += search_for_wahlRoots1(child)
    return wplist


xml = read_xml(path = "MDB_STAMMDATEN.XML")

WProots = search_for_wahlRoots1(xml)
    
#Nunn müssen noch die gewünschten Nummern aus dem Code extrahiert werden    

def extracting_WP(list1):
    WPelements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        WPelements += [sub_list]
    return WPelements

WPs = extracting_WP(WProots)


#Testen der neuen Liste auf längenmäßige Kompatibilität mit dem DF  
print(WPs[:4])
print(len(WPs))

[['5', '6', '7', '8', '9', '10', '11'], ['3', '4', '5', '6', '7'], ['2', '3', '4'], ['11', '12']]
4073


In [11]:
#Erweiterung
df_first["WAHLPERIODEN"] = WPs

df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]"
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]"
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]"


In [12]:
#Der gleiche Prozess wird nun auch für die Manadatsart gewählt. In der Liste die nun bei jedem MdB steht, kann man die Mandatsart
#zur jeweiligen Wahlperiode(gleicher Index) finden.

def search_for_MandatsArtRoots1(root):
    
    #nested function (Entscheidend ist hier der Break beim nächsten "ID" tag)
    def search_for_MandatRoots2(root2):
        mandatlist2 = []
        for child2 in root2:
            if child2.tag == "MANDATSART":
                mandatlist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                mandatlist2 += search_for_MandatRoots2(child2)
        return mandatlist2
    #ende nested function
    
    mandatlist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            mandatlist += [search_for_MandatRoots2(child)]
        else:
            mandatlist += search_for_MandatsArtRoots1(child)
    return mandatlist


# Schon weiter oben passiert: xml = read_xml(path = "MDB_STAMMDATEN.XML")

MandatRoots = search_for_MandatsArtRoots1(xml)
    
#Nun müssen noch die gewünschten Mandatsarten aus dem Code extrahiert werden    

def extracting_Mandate(list1):
    MandatElements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        MandatElements += [sub_list]
    return MandatElements

Mandate = extracting_Mandate(MandatRoots)


#Testen der neuen Liste auf längenmäßige Kompatibilität mit dem DF  
print(Mandate[:4])
print(len(Mandate))

[['Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl'], ['Landesliste', 'Landesliste', 'Landesliste', 'Landesliste', 'Landesliste'], ['Landesliste', 'Landesliste', 'Landesliste'], ['Volkskammer', 'Landesliste']]
4073


In [13]:
#Einfügen ins DF
df_first["MANDATSARTEN"] = Mandate
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]","[Landesliste, Landesliste, Landesliste, Landes..."
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]","[Landesliste, Landesliste, Landesliste]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]","[Volkskammer, Landesliste]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."


In [14]:
#Im folgenden wird experimentiert und umgestaltet, daher legen wir zunächst eine Kopie an

dfnew = df_first.copy()

In [15]:
#Kurzer Check für das Replacing im nchsten Schritt
dfnew['GESCHLECHT'].unique()

array(['männlich', 'weiblich'], dtype=object)

In [16]:
#Um unser DF analysierbarer zu machen legen wir nun boolean-Spalten an.
dfnew["AUSLAND"] = dfnew["GEBURTSLAND"] != "Deutschland"
dfnew['AUSLAND'].replace([True, False], [1,0], inplace = True)
#Hier ist es natürlich nicht ganz akkurat Migrierte mit im II. WK vertriebenen gleichzusetzten, aber, diesem Effekt können wir
#noch mehr Beachtung schenken

#Ähnliche umgestaltung auch für die Spalte mt dem Geschlecht
dfnew['GESCHLECHT'].replace(["männlich", "weiblich"],[1,0], inplace = True)



In [17]:
print(df_first.head())
print(dfnew["AUSLAND"].head())
print(dfnew['GESCHLECHT'].head())

         ID GESCHLECHT PARTEI_KURZ GEBURTSDATUM  \
0  11000001   männlich         CDU         1930   
1  11000002   männlich         FDP         1909   
2  11000003   weiblich         CDU         1913   
3  11000004   weiblich         CDU         1933   
4  11000005   männlich         CDU         1950   

                                               BERUF  GEBURTSLAND   NACHNAME  \
0  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...  Deutschland    Abelein   
1                             Rechtsanwalt und Notar  Deutschland  Achenbach   
2                                    Hilfsreferentin  Jugoslawien  Ackermann   
3                                             Ärztin  Deutschland  Ackermann   
4                      Mathematiker, Geschäftsführer  Deutschland       Adam   

     VORNAME             WAHLPERIODEN  \
0    Manfred  [5, 6, 7, 8, 9, 10, 11]   
1      Ernst          [3, 4, 5, 6, 7]   
2  Annemarie                [2, 3, 4]   
3       Else                 [11, 12]   
4     U

In [29]:
#Für die Pivotisierung wollen wir jetzt 19 Columns für die verschiedenen Wahlperioden erstellen und diese mit elementen aus [1,0] besetzen.
#Wie viele Wahlperioden gibt es?
unique_list = []
for e in dfnew["WAHLPERIODEN"]:
    for i in e:
        if int(i) in unique_list:
            None
        else:
            unique_list.append(int(i))
            
unique_list.sort()            
print(unique_list)
        

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [31]:
#Erstellen der 19 Spalten

def WPcolumnchecker(column, range1):
    
    #Diese Funktion soll eine iteration über die Listen in der Spalte "Wahlperioden" durchführen
    def listchecker(WPNum, WPList):
        if WPNum in WPList:
            return 1
        else:
            return 0
    
    for n in range1:
        WPstr = "WP" + str(n)
        emp_list = []
        nstr= str(n)
        for index in range(0,4073):
            WPListe = column.loc[index]
            emp_list.append(listchecker(nstr, WPListe))
        dfnew[WPstr] = emp_list
        
    
        
dfrange1 = unique_list
dfcolumn = dfnew["WAHLPERIODEN"]

WPcolumnchecker(dfcolumn, dfrange1)
            

In [34]:
#Überprüfung für ein Beispiel --> sind die 1 und 0 an den richtigen Stellen
print(dfnew.loc[1])

ID                                                       11000002
GESCHLECHT                                                      1
PARTEI_KURZ                                                   FDP
GEBURTSDATUM                                                 1909
BERUF                                      Rechtsanwalt und Notar
GEBURTSLAND                                           Deutschland
NACHNAME                                                Achenbach
VORNAME                                                     Ernst
WAHLPERIODEN                                      [3, 4, 5, 6, 7]
MANDATSARTEN    [Landesliste, Landesliste, Landesliste, Landes...
AUSLAND                                                         0
WP1                                                             0
WP2                                                             0
WP3                                                             1
WP4                                                             1
WP5       

In [50]:
#aWenn irgendwer Ahnung hat, wie man Namen von Variablen generieren kann, BITTE ÄNDERN!

df_WP1 = dfnew.loc[dfnew["WP1"] == 1]
df_WP2 = dfnew.loc[dfnew["WP2"] == 1]
df_WP3 = dfnew.loc[dfnew["WP3"] == 1]
df_WP4 = dfnew.loc[dfnew["WP4"] == 1]
df_WP5 = dfnew.loc[dfnew["WP5"] == 1]
df_WP6 = dfnew.loc[dfnew["WP6"] == 1]
df_WP7 = dfnew.loc[dfnew["WP7"] == 1]
df_WP8 = dfnew.loc[dfnew["WP8"] == 1]
df_WP9 = dfnew.loc[dfnew["WP9"] == 1]
df_WP10 = dfnew.loc[dfnew["WP10"] == 1]
df_WP11 = dfnew.loc[dfnew["WP11"] == 1]
df_WP12 = dfnew.loc[dfnew["WP12"] == 1]
df_WP13 = dfnew.loc[dfnew["WP13"] == 1]
df_WP14 = dfnew.loc[dfnew["WP14"] == 1]
df_WP15 = dfnew.loc[dfnew["WP15"] == 1]
df_WP16 = dfnew.loc[dfnew["WP16"] == 1]
df_WP17 = dfnew.loc[dfnew["WP17"] == 1]
df_WP18 = dfnew.loc[dfnew["WP18"] == 1]
df_WP19 = dfnew.loc[dfnew["WP19"] == 1]



In [105]:
df_WP1[:-2].head()
#df_WP1.describe()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
7,11000009,1,CDU,1876,Bundeskanzler a. D.,Deutschland,Adenauer,Konrad,"[1, 2, 3, 4, 5]","[Direktwahl, Direktwahl, Direktwahl, Direktwah...",...,0,0,0,0,0,0,0,0,0,0
11,11000013,1,KPD,1904,"Bergmann, Gewerkschaftsfunktionär",Deutschland,Agatz,Willi,[1],[Landesliste],...,0,0,0,0,0,0,0,0,0,0
13,11000015,1,DP,1879,Kapitän a. D.,Deutschland,Ahrens,Adolf,[1],[Landesliste],...,0,0,0,0,0,0,0,0,0,0
18,11000020,1,CDU,1890,Geschäftsführer,Deutschland,Albers,Johannes,"[1, 2]","[Direktwahl, Direktwahl]",...,0,0,0,0,0,0,0,0,0,0
19,11000021,0,SPD,1901,Buchhalterin,Deutschland,Albertz,Luise,"[1, 2, 3, 4, 5]","[Landesliste, Landesliste, Landesliste, Landes...",...,0,0,0,0,0,0,0,0,0,0


In [52]:
df_WP19.describe()
#das sieht nicht gut aus --> zwei Pateiwechsler von der AfD werden doppelt gezählz (709 müssten eig. sein)

,GESCHLECHT,AUSLAND,WP1,WP2,WP3,WP4,WP5,WP6,WP7,WP8,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
count,711.000000,711.000000,711.0,711.0,711.0,711.0,711.0,711.0,711.000000,711.000000,...,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.0
mean,0.690577,0.016878,0.0,0.0,0.0,0.0,0.0,0.0,0.001406,0.001406,...,0.002813,0.007032,0.021097,0.042194,0.091421,0.158931,0.232068,0.407876,0.592124,1.0
std,0.462581,0.128904,0.0,0.0,0.0,0.0,0.0,0.0,0.037503,0.037503,...,0.053000,0.083623,0.143809,0.201173,0.288409,0.365869,0.422449,0.491786,0.491786,0.0
min,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
75%,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0
max,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
